In [1]:
import pandas as pd

### 지도학습

#### 분류 - 이진분류
- 목표변수: 범주형 (0 or 1)
- 설명변수: 


In [22]:
df_TFD = pd.read_csv('../../../datasets/TitanicFromDisaster_train.csv')
df_TFD

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


#### data preprocessing _ 전처리
- 목표변수: Survived (범주형) 
- 설명변수: Pclass (범주형), Age(연속형)

In [3]:
df_TFD.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
df_TFD_extract = df_TFD[['Survived', 'Pclass','Age' ]]

In [5]:
df_TFD_extract.isnull().sum()

Survived      0
Pclass        0
Age         177
dtype: int64

In [6]:
df_TFD_extract_preprocess = df_TFD_extract.dropna()
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age
0,0,3,22.0
1,1,1,38.0


#### structured data / 정형화 단계
- 7:3으로 data를 나눌거다.
   - from sklearn.model_selection import train_test_split
   - features_train, features_test, target_train, target_test = train_test_split(features, target, random_state =111 ← random_state 로 고정을 한다.  )
   - 500개 이하는 split NO NO NO. 

In [19]:
from sklearn.model_selection import train_test_split
target = df_TFD_extract_preprocess['Survived']
features = df_TFD_extract_preprocess[['Pclass','Age']]


In [27]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state= 111)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((535, 2), (179, 2), (535,), (179,))

#### Learning _모델 학습

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
model = LogisticRegression()

In [30]:
model.fit(features_train,target_train)

LogisticRegression()

###### Logistic Regression 
y = 1/(1+e^-(wx+b))

In [31]:
model.coef_, model.intercept_
#이 공식은 logistics Regression 의 공식의 각 변수들의 상수임. 

(array([[-1.25703902, -0.04521969]]), array([3.78621796]))

#### 예측

In [65]:
df_TFD_extract_preprocess[250:255] # index가 features_train 순서와 동일함. 

,Survived,Pclass,Age
313,0,3,28.0
314,0,2,43.0
315,1,3,26.0
316,1,2,24.0
317,0,2,54.0


In [44]:
model.predict(features_train[250:255])

array([1, 1, 1, 1, 0], dtype=int64)

- model.predict_proba(features_train[250:255]) : 확률값이 나옴. 

In [45]:
model.predict_proba(features_train[250:255])

array([[0.40038684, 0.59961316],
       [0.2795913 , 0.7204087 ],
       [0.08026591, 0.91973409],
       [0.17735874, 0.82264126],
       [0.60973734, 0.39026266]])

##### 평가
- ** 실무에선 사용 안함. 
- from sklearn.metrics import accuracy_score 

In [56]:
target_train_predict = model.predict(features_train)
target_train_predict.shape # target_train.shape과 동일

(535,)

In [66]:
from sklearn.metrics import accuracy_score # 정확도를 평가함

In [67]:
accuracy_score(target_train, target_train_predict) # 내부 평가

0.7065420560747664

In [68]:
target_test_predict = model.predict(features_test)
target_test_predict.shape # target_test.shape 동일

(179,)

In [69]:
accuracy_score(target_test, target_test_predict) # 외부 평가

0.659217877094972

- accuracy_score(target_train, target_train_predict) # 내부 평가 _ 0.7065420560747664 
- accuracy_score(target_test, target_test_predict) # 외부 평가 _ 0.659217877094972

#### 평가 주요 사용 
- from sklearn.metrics import classification_report _ 종합적으로 나옴
    - precision, recall, f1-score, support : 이 중에 f1-score를 중요하게 볼 것. (판단에 필요한 score) 
    - survived의 0, 1의 일치하는 score임_각 항목중 f1-score를 보고 판단함.  
    - macro avg를 볼 것. : macro avg는 0,1 의 각 항목 평균값을 나타냄. precision과 recall과 f1-score은 macro avg를 주요하게 봄.
    

In [70]:
from sklearn.metrics import classification_report

In [71]:
print (classification_report(target_train, target_train_predict))

              precision    recall  f1-score   support

           0       0.71      0.83      0.77       312
           1       0.69      0.54      0.60       223

    accuracy                           0.71       535
   macro avg       0.70      0.68      0.69       535
weighted avg       0.70      0.71      0.70       535



In [72]:
print (classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.73      0.72      0.73       112
           1       0.54      0.55      0.55        67

    accuracy                           0.66       179
   macro avg       0.64      0.64      0.64       179
weighted avg       0.66      0.66      0.66       179



test예측값은 train 값보다 f1-score의 0과 1 사이가 조금 차이남. 